-----
# Simple Two Player Games
-----

# 1. The Problem

As we explore this area and try to find driving factors that assist in developing an optimal strategy for various two player games, we must first address some concerns and develop a plan of execution. This will ultimately help us know where the project and research have been, are currently, and are going to head towards. 

# 2. Recap Fall 2020
During this semester, we all met and discussed the concepts the make up a game and what characteristics are to be observed to assess if there is a winner. This led to the creation of several python game classes that would be used to flesh out, what I will now call, a proof-of-concept for Toothpick Takeaway. By October/November, we were able to finally begin collecting data to start some very primitive analysis. One of the issues we faced was the formatting of the data in our final CSV as larger games did not format quite as nicely (this will be address later in this document).

## 2.1 Results

A great thing that came from this endeavor was how we were able to abstract out the game glasses enough to let them act as interfaces for each *new* game. These files can be located along the path `/simple_games/generic_classes/`. Inside are the components that make up every game: 
- board - where the game is being played
- player - represents a single player in a game
- referee - monitors the moves of a player and validates attempted moves
- game  - the object that has all of the mentioned components and begins the game

Another great skill we developed over the semester was the ability to work as a team, communicating with each other via email and other means as necessary, to see the project progress. Our utilization of GitHub issues allowed for each member to choose something to work on. This helped keep a steady workflow and followed some of the principles of Agile development like Dr. Penland was hoping. 

---
# 3. Spring 2021
----

## 3.1 Progress

This brings us to this semester. We have been able to further refine how the generic classes work and introduced some interfaces for our `ruleset` and `strategy` for each game. Interfaces allow us to require a certain functionality but the implementation is left to be decided for each game. 

Note: Under [Program Requirements](#Program-Requirements), we will address the interfaces and why they matter in more detail.

Our goals this semester include developing software that allows for new games to be added easily (which we are almost there) to gather data from in hopes of designing a genetic algorithm that can start to evolve against our hard coded strategies. Another goal, more in the immediate is to prepare a small presentation of all of the research up to this point for the MAA SE conference. 

The second is doable and while the second is more involved, can be achieved relatively soon. 

## 3.2 Overcoming Issues
One of the biggest issues that we are currently dealing with is identifying the system the this project relies on to function and how to design components such that we can each build something and then integrate back together. 

We plan to address this concern at the next meeting so that everyone can feel a sense of project security. 

---
# 4. Development Refinement
-----

## 4.1 Program Requirements

A Two Player Game must have some baseline functionality to operate. This functionality is met by properly implemented the required interfaces on a new `Ruleset` class and `Strategy` class.

## 4.2 Why use interfaces?

The use of the interface allows for each new game to still utilize the `generic` classes that we created last semester and implement the functionality as prescribed by `ruleset` and `strategy` interface. In other words, our program will be easier to scale upwards since the implementation details for the strategy and rule sets are dependent on *each new game*.

### 4.2.1 The Ruleset

A ruleset is the set of rules that define how a game operates. Creating a ruleset involves implementing the required functions provided by the `RulesetInterface`. The functions consist of the following:

* `__init__(name, initial_state, bounds = None)`: A constructor to setup the ruleset. A ruleset name (usually the game's name) must be provided. Additionally, the initial state of                                                    the game board must be specified. If necessary, the bounds of the game board may be provided. Think critically about what kind of                                                      board the game will need. For example, a game needing a tileboard needs bounds to determine its size.

* `is_legal(board, proposed_move)`: Determines if the proposed move is legal given the current board state. This function does not need to know who made the move, as it only                                             determines whether the move was legal or not. It **must** return either `True` or `False`.


* `is_game_over(board)`: Determines if the game is over. This function must analyze the current state of the board to see if the end condition has been met. If the board is in an end                          state, the game is over. This function **must** return either `True` or `False`.

* `update_board(board, player, move)`: Updates the board to reflect the move made. Also updates the board's move history logger, which is by default `board.data`. The index of the                                          move history should be a stringified representation of the board's state and the value at that index should be a tuple containing the player's                                        name and the move made.

### 4.2.2 The Strategy

A strategy determines how a player will make a move on the board. Creating a strategy involves implementing the required functions provided by the `StrategyInterface`. The functions consist of the following:

* `__init__(name, data = None)`:  A constructor to setup the strategy. A strategy name must be provided. If desired, data can be provided to the strategy.
* `move(board)`: Make a move on the given board based on the implemented strategy.

## 4.3 Constraints

No additional parameters may be passed into any of the functions. Additional functions may be added to each implementation, but all original functions must be implemented.

Be mindful of how game data will be collected. The `board` has a `data` field that is a dictionary where the keys are strings representing the board state at the current move and the values are tuples containing the player's name and the move made at that move.

## 4.4 Instructions

All necessary files need to be imported, which can become cumbersome depending on where the game is being constructed relative to the necessary files.

Once all files are imported, the game can be constructed and ran with the following code:

---------
# 5. Example Implementation
---------

Below is an example implementation of a very simple guessing game game- We'll just call it *Guess*.

The rules are simple: A random number is chosen to start. Both players alternate trying to guess the number. Players win when either they correctly guess the number number or their opponent guesses a previous-guessed number.

#### We begin by implementing the ruleset:

In [1]:
import random
from generic_classes.ruleset_interface import RulesetInterface

class GuessRuleset(RulesetInterface):
    def __init__(self, name, initial_state = 5, bounds = 10):
        """
        Rulset constructor.
        
        Args:
            name : The name of the ruleset
            initial_state : The winning number
            bounds : The losing number
        """
        self.name = name
        self.initial_state = initial_state
        self.bounds = bounds
    
    def is_legal(self, board, proposed_move):
        """
        Determines if the move proposed is legal.
        
        Args:
            board : The board being played on
            proposed_move : The move being proposed
        
        Return:
            True if the guess was between 0 and the boards bounds and has not yet been guessed.
        """
        return proposed_move >= 0 and proposed_move <= board.bounds and proposed_move not in list(board.data.keys())

    def is_game_over(self, board):
        """
        Determines if the game is over.
        
        Args:
            board : The board being played on
            
        Return:
            True if the guessed number is the winning or losing number.
        """
        return board.state == self.bounds or board.state == self.initial_state
    
    def update_board(self, board, player, move):
        """
        Updates board's state to reflect the current guess.
        
        Args:
            board : The board being played on
            player : The player who made the move
            move : The move being made
        """
        board.state = move
        board.data[board.state] = (player.name, move)

#### Next, we implement a strategy for each player to use:

In [2]:
import random
from generic_classes.strategy_interface import StrategyInterface

class GuessStrategy(StrategyInterface):

    def __init__(self, name, data = None):
        """
        Strategy constructor.

        Args:
            name : The name of the strategy
            data : Data to read from, if applicable
        """
        self.name = name
        self.data = data

    def move(self, board):
        """
        Randomly pick a number.
        
        Args:
            board : The board being played on

        Return:
            A random number between 0 and the board's bounds.
        """
        return random.randrange(0, board.bounds + 1)

#### Now, assemble it all together

In [3]:
from generic_classes.player import Player
from generic_classes.board import Board
from generic_classes.referee import Referee
from generic_classes.game import Game

In [8]:
ruleset = GuessRuleset("Basic Guess Ruleset", 5, 10)
strategy = GuessStrategy("Random Strategy for Guess")

p1 = Player("player1", strategy)
p2 = Player("player2", strategy)

board = Board(ruleset.initial_state, ruleset.bounds)

ref = Referee(board, ruleset)

game = Game(ref, board, [p1, p2])

board = game.play()

View the game data by accessing the `board.data` variable

----
# 6. Running the Game
---

In [9]:
board.data

{2: ('player1', 2),
 6: ('player2', 6),
 9: ('player1', 9),
 5: ('player2', 5),
 'winner': 'player2'}

Note: This example game is not flawless or very complex. However, it has enough restrictions that it showcases how to implement all necessary files properly.